In [49]:
import pandas as pd
import numpy as np
import requests
import json
import mercury as mr
from opensearchpy import OpenSearch
import ast

In [4]:
df = pd.read_csv('../data/vgenome_sample_1k.csv', index_col=0)
df = df.rename(columns={'tags': 'tags_list'})
df['tags_list'] = df['tags_list'].apply(ast.literal_eval)
df['tags'] = df['tags_list'].apply(lambda x: " ".join(x))
df.head()

,image_id,url,tags_list,tags
0,1,https://cs.stanford.edu/people/rak248/VG_100K_...,"[windows, car, man, trees, back, tree trunk, a...",windows car man trees back tree trunk arm bike...
1,2,https://cs.stanford.edu/people/rak248/VG_100K/...,"[sign, sidewalk, window, backpack, sneakers, t...",sign sidewalk window backpack sneakers tree bu...
2,3,https://cs.stanford.edu/people/rak248/VG_100K/...,"[photos, wireless phone, chain, computer tower...",photos wireless phone chain computer tower hai...
3,4,https://cs.stanford.edu/people/rak248/VG_100K/...,"[carpet, chair, cloths, colour, frame, drape, ...",carpet chair cloths colour frame drape table t...
4,5,https://cs.stanford.edu/people/rak248/VG_100K/...,"[windows, chair, colour, paper, cables, color,...",windows chair colour paper cables color shelf ...


In [89]:
# create the judgement data

# let's first choose 10 tags that we will search images for
all_tags = []

for tag in df['tags_list'].values:
    all_tags += tag

#all_tags = (all_tags))
len(all_tags)

14347

In [96]:
from collections import Counter

occurrences = pd.Series(all_tags).value_counts()
occurrences[:10]

building    385
window      340
wall        283
man         226
tree        209
sign        203
car         198
table       178
person      175
sky         174
Name: count, dtype: int64

In [114]:
# check
ct = 0
for i, row in df.iterrows():
    for tag in set(row['tags_list']):
        if tag == 'man':
            print(row['image_id'])
            ct += 1

ct

1
2
8
10
16
18
20
23
24
25
26
45
47
48
49
57
58
59
61
65
70
74
77
83
87
105
109
114
115
126
131
133
138
141
153
154
155
156
167
171
172
176
178
192
210
220
235
243
248
252
260
264
268
277
280
287
304
305
314
319
323
326
338
339
341
344
345
355
360
361
365
370
386
390
393
396
397
399
400
402
407
408
410
411
415
416
418
419
420
426
427
436
440
443
448
449
452
455
456
464
465
473
487
515
569
576
585
587
588
590
591
592
594
596
597
598
604
608
610
611
615
617
620
622
623
624
626
630
632
633
635
637
638
639
640
644
651
652
654
664
667
668
671
674
675
684
692
712
722
723
724
731
734
735
736
744
751
768
769
770
791
795
800
804
807
812
817
818
829
840
857
881
883
884
887
891
901
904
906
910
911
918
922
925
930
940
947
949
950
953
961
963
964
965
971
983
988
990
994
998
1003
1011
1013
1015
1017
1018
1020
1021
1022
1023
1027
1029
1035
1037
1038
1042
1043
1047
1048
1049
1055
1069
1071
1077
1079
1080


226

In [101]:
select_tags = list(occurrences[:10].index.values)
select_tags

['building',
 'window',
 'wall',
 'man',
 'tree',
 'sign',
 'car',
 'table',
 'person',
 'sky']

In [103]:
import random

random_tags = random.sample(all_tags, 3)
random_tags

['hair', 'conference table', 'tree']

In [104]:
select_tags = list(set(select_tags + random_tags))
select_tags

['man',
 'sign',
 'sky',
 'tree',
 'person',
 'table',
 'hair',
 'conference table',
 'wall',
 'building',
 'window',
 'car']

In [118]:
judgement_data = {}

for search_tag in select_tags:
    matched_images = []
    for i, row in df.iterrows():
        for available_tag in set(row['tags_list']):
            if available_tag == search_tag:
                matched_images.append(row['image_id'])
    judgement_data[search_tag] = matched_images

In [121]:
df_judgement = pd.DataFrame(list(judgement_data.items()), columns=['search_tag', 'image_ids'])
df_judgement

,search_tag,image_ids
0,man,"[1, 2, 8, 10, 16, 18, 20, 23, 24, 25, 26, 45, ..."
1,sign,"[1, 2, 23, 26, 30, 47, 48, 49, 54, 57, 58, 59,..."
2,sky,"[22, 25, 26, 29, 45, 49, 52, 87, 126, 132, 133..."
3,tree,"[1, 2, 16, 23, 26, 38, 48, 52, 53, 87, 101, 10..."
4,person,"[21, 23, 25, 40, 53, 54, 55, 57, 72, 83, 108, ..."
5,table,"[3, 4, 10, 11, 25, 27, 28, 30, 33, 34, 35, 36,..."
6,hair,"[3, 7, 16, 18, 20, 21, 26, 29, 65, 71, 72, 77,..."
7,conference table,"[348, 1075]"
8,wall,"[1, 3, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17,..."
9,building,"[1, 2, 9, 11, 16, 22, 23, 24, 25, 26, 29, 42, ..."


In [122]:
df_judgement['matched_image_ct'] = df_judgement['image_ids'].apply(lambda x: len(x))
df_judgement

,search_tag,image_ids,matched_image_ct
0,man,"[1, 2, 8, 10, 16, 18, 20, 23, 24, 25, 26, 45, ...",226
1,sign,"[1, 2, 23, 26, 30, 47, 48, 49, 54, 57, 58, 59,...",203
2,sky,"[22, 25, 26, 29, 45, 49, 52, 87, 126, 132, 133...",174
3,tree,"[1, 2, 16, 23, 26, 38, 48, 52, 53, 87, 101, 10...",209
4,person,"[21, 23, 25, 40, 53, 54, 55, 57, 72, 83, 108, ...",175
5,table,"[3, 4, 10, 11, 25, 27, 28, 30, 33, 34, 35, 36,...",178
6,hair,"[3, 7, 16, 18, 20, 21, 26, 29, 65, 71, 72, 77,...",78
7,conference table,"[348, 1075]",2
8,wall,"[1, 3, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17,...",283
9,building,"[1, 2, 9, 11, 16, 22, 23, 24, 25, 26, 29, 42, ...",385


In [53]:
url = "http://localhost:9200/_plugins/_ml/models/_search"

headers = {
    'Content-Type': 'application/json'
}

payload = {
  "query": {
    "match_all": {}
  },
  "size": 1
}

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

model_id = response.json()['hits']['hits'][0]['_source']['model_id']
model_id

'zn6HYZQBrLHGb1uDUQni'

In [56]:
url = "http://localhost:9200/_search/pipeline/default_model_pipeline"

headers = {
    'Content-Type': 'application/json'
}

payload = {
  "request_processors": [
    {
      "neural_query_enricher" : {
        "default_model_id": f"{model_id}",
      }
    }
  ]
}

response = requests.put(url, headers=headers, data=json.dumps(payload))

In [57]:
mr.JSON(response.json(), level=4)

In [69]:
url = "http://localhost:9200/tags_db/_search"

payload = {
  "_source": {
    "excludes": [
      "tag_embedding"
    ]
  },
  "query": {
    "neural": {
      "tag_embedding": {
        "query_text": "circle",
        "model_id": f"{model_id}",
        "k": 5
      }
    }
  }
}

In [70]:
response = requests.get(url, headers=headers, data=json.dumps(payload))

mr.JSON(response.json(), level=4)

In [80]:
response.json()['hits']['hits']

[{'_index': 'tags_db',
  '_id': '157',
  '_score': 0.4294047,
  '_source': {'tags': 'tree wall'}},
 {'_index': 'tags_db',
  '_id': '188',
  '_score': 0.42106822,
  '_source': {'tags': 'bar cloud sign sand swing set tree pole palm background ground shadow park sky ring wall'}},
 {'_index': 'tags_db',
  '_id': '24',
  '_score': 0.40440634,
  '_source': {'tags': 'busy pole car traffic poles man lighting stripes motor bike motorcycles bicycle motorbike trees on sidewalk scooter bikes lines in distance marks next to street street bottle photo building pvement boy'}},
 {'_index': 'tags_db',
  '_id': '826',
  '_score': 0.4032276,
  '_source': {'tags': 'tree post'}},
 {'_index': 'tags_db',
  '_id': '140',
  '_score': 0.40300167,
  '_source': {'tags': 'sign sidewalk pole bike fence wall'}}]

In [81]:
matched_image_ids = [i['_id'] for i in response.json()['hits']['hits']]
matched_image_ids

['157', '188', '24', '826', '140']

In [130]:
num_search_results = 10

for i, row in df_judgement.iterrows():
    search_tag = row['search_tag']
    payload = {
      "_source": {
        "excludes": [
          "tag_embedding"
        ]
      },
      "query": {
        "neural": {
          "tag_embedding": {
            "query_text": search_tag,
            "model_id": f"{model_id}",
            "k": num_search_results
          }
        }
      }
    }

    response = requests.get(url, headers=headers, data=json.dumps(payload))

    matched_image_ids = [i['_id'] for i in response.json()['hits']['hits']]

    print(matched_image_ids)
    df_judgement.loc[i, 'search_results'] = str(matched_image_ids)

['868', '264', '420', '109', '591', '594', '448', '23', '300', '58']
['995', '300', '596', '1003', '478', '355', '804', '58', '140', '54']
['781', '663', '863', '812', '896', '838', '189', '651', '478', '648']
['826', '157', '844', '218', '972', '951', '1025', '766', '204', '221']
['481', '781', '929', '424', '880', '867', '51', '273', '342', '764']
['1044', '535', '1054', '551', '855', '172', '567', '35', '550', '560']
['1018', '21', '167', '696', '1055', '277', '260', '412', '396', '419']
['1044', '535', '1054', '200', '551', '855', '491', '404', '172', '1007']
['157', '995', '607', '720', '1044', '322', '144', '253', '285', '81']
['56', '577', '649', '728', '774', '776', '981', '880', '602', '432']
['56', '577', '649', '728', '774', '776', '981', '607', '720', '432']
['868', '429', '863', '953', '768', '113', '612', '625', '530', '203']


In [131]:
df_judgement

,search_tag,image_ids,matched_image_ct,search_results,common_count
0,man,"[1, 2, 8, 10, 16, 18, 20, 23, 24, 25, 26, 45, ...",226,"['868', '264', '420', '109', '591', '594', '44...",5
1,sign,"[1, 2, 23, 26, 30, 47, 48, 49, 54, 57, 58, 59,...",203,"['995', '300', '596', '1003', '478', '355', '8...",3
2,sky,"[22, 25, 26, 29, 45, 49, 52, 87, 126, 132, 133...",174,"['781', '663', '863', '812', '896', '838', '18...",5
3,tree,"[1, 2, 16, 23, 26, 38, 48, 52, 53, 87, 101, 10...",209,"['826', '157', '844', '218', '972', '951', '10...",4
4,person,"[21, 23, 25, 40, 53, 54, 55, 57, 72, 83, 108, ...",175,"['481', '781', '929', '424', '880', '867', '51...",4
5,table,"[3, 4, 10, 11, 25, 27, 28, 30, 33, 34, 35, 36,...",178,"['1044', '535', '1054', '551', '855', '172', '...",5
6,hair,"[3, 7, 16, 18, 20, 21, 26, 29, 65, 71, 72, 77,...",78,"['1018', '21', '167', '696', '1055', '277', '2...",5
7,conference table,"[348, 1075]",2,"['1044', '535', '1054', '200', '551', '855', '...",0
8,wall,"[1, 3, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17,...",283,"['157', '995', '607', '720', '1044', '322', '1...",5
9,building,"[1, 2, 9, 11, 16, 22, 23, 24, 25, 26, 29, 42, ...",385,"['56', '577', '649', '728', '774', '776', '981...",5


In [132]:
df_judgement["search_results"] = df_judgement["search_results"].apply(lambda x: list(map(int, ast.literal_eval(x))))
df_judgement

,search_tag,image_ids,matched_image_ct,search_results,common_count
0,man,"[1, 2, 8, 10, 16, 18, 20, 23, 24, 25, 26, 45, ...",226,"[868, 264, 420, 109, 591, 594, 448, 23, 300, 58]",5
1,sign,"[1, 2, 23, 26, 30, 47, 48, 49, 54, 57, 58, 59,...",203,"[995, 300, 596, 1003, 478, 355, 804, 58, 140, 54]",3
2,sky,"[22, 25, 26, 29, 45, 49, 52, 87, 126, 132, 133...",174,"[781, 663, 863, 812, 896, 838, 189, 651, 478, ...",5
3,tree,"[1, 2, 16, 23, 26, 38, 48, 52, 53, 87, 101, 10...",209,"[826, 157, 844, 218, 972, 951, 1025, 766, 204,...",4
4,person,"[21, 23, 25, 40, 53, 54, 55, 57, 72, 83, 108, ...",175,"[481, 781, 929, 424, 880, 867, 51, 273, 342, 764]",4
5,table,"[3, 4, 10, 11, 25, 27, 28, 30, 33, 34, 35, 36,...",178,"[1044, 535, 1054, 551, 855, 172, 567, 35, 550,...",5
6,hair,"[3, 7, 16, 18, 20, 21, 26, 29, 65, 71, 72, 77,...",78,"[1018, 21, 167, 696, 1055, 277, 260, 412, 396,...",5
7,conference table,"[348, 1075]",2,"[1044, 535, 1054, 200, 551, 855, 491, 404, 172...",0
8,wall,"[1, 3, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17,...",283,"[157, 995, 607, 720, 1044, 322, 144, 253, 285,...",5
9,building,"[1, 2, 9, 11, 16, 22, 23, 24, 25, 26, 29, 42, ...",385,"[56, 577, 649, 728, 774, 776, 981, 880, 602, 432]",5


In [133]:
df_judgement["common_count"] = df_judgement.apply(lambda row: \
                                                  len(set(row["image_ids"]) & set(row["search_results"])), axis=1)

df_judgement

,search_tag,image_ids,matched_image_ct,search_results,common_count
0,man,"[1, 2, 8, 10, 16, 18, 20, 23, 24, 25, 26, 45, ...",226,"[868, 264, 420, 109, 591, 594, 448, 23, 300, 58]",8
1,sign,"[1, 2, 23, 26, 30, 47, 48, 49, 54, 57, 58, 59,...",203,"[995, 300, 596, 1003, 478, 355, 804, 58, 140, 54]",8
2,sky,"[22, 25, 26, 29, 45, 49, 52, 87, 126, 132, 133...",174,"[781, 663, 863, 812, 896, 838, 189, 651, 478, ...",10
3,tree,"[1, 2, 16, 23, 26, 38, 48, 52, 53, 87, 101, 10...",209,"[826, 157, 844, 218, 972, 951, 1025, 766, 204,...",8
4,person,"[21, 23, 25, 40, 53, 54, 55, 57, 72, 83, 108, ...",175,"[481, 781, 929, 424, 880, 867, 51, 273, 342, 764]",8
5,table,"[3, 4, 10, 11, 25, 27, 28, 30, 33, 34, 35, 36,...",178,"[1044, 535, 1054, 551, 855, 172, 567, 35, 550,...",10
6,hair,"[3, 7, 16, 18, 20, 21, 26, 29, 65, 71, 72, 77,...",78,"[1018, 21, 167, 696, 1055, 277, 260, 412, 396,...",10
7,conference table,"[348, 1075]",2,"[1044, 535, 1054, 200, 551, 855, 491, 404, 172...",0
8,wall,"[1, 3, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17,...",283,"[157, 995, 607, 720, 1044, 322, 144, 253, 285,...",7
9,building,"[1, 2, 9, 11, 16, 22, 23, 24, 25, 26, 29, 42, ...",385,"[56, 577, 649, 728, 774, 776, 981, 880, 602, 432]",10


In [138]:
df_judgement['precision'] = df_judgement['common_count'] / num_search_results

df_judgement[f'recall_at_{num_search_results}'] = df_judgement['common_count'] / df_judgement['matched_image_ct'] 
df_judgement

,search_tag,image_ids,matched_image_ct,search_results,common_count,precision_at_10,recall_at_10,precision
0,man,"[1, 2, 8, 10, 16, 18, 20, 23, 24, 25, 26, 45, ...",226,"[868, 264, 420, 109, 591, 594, 448, 23, 300, 58]",8,0.035398,0.035398,0.8
1,sign,"[1, 2, 23, 26, 30, 47, 48, 49, 54, 57, 58, 59,...",203,"[995, 300, 596, 1003, 478, 355, 804, 58, 140, 54]",8,0.039409,0.039409,0.8
2,sky,"[22, 25, 26, 29, 45, 49, 52, 87, 126, 132, 133...",174,"[781, 663, 863, 812, 896, 838, 189, 651, 478, ...",10,0.057471,0.057471,1.0
3,tree,"[1, 2, 16, 23, 26, 38, 48, 52, 53, 87, 101, 10...",209,"[826, 157, 844, 218, 972, 951, 1025, 766, 204,...",8,0.038278,0.038278,0.8
4,person,"[21, 23, 25, 40, 53, 54, 55, 57, 72, 83, 108, ...",175,"[481, 781, 929, 424, 880, 867, 51, 273, 342, 764]",8,0.045714,0.045714,0.8
5,table,"[3, 4, 10, 11, 25, 27, 28, 30, 33, 34, 35, 36,...",178,"[1044, 535, 1054, 551, 855, 172, 567, 35, 550,...",10,0.056180,0.056180,1.0
6,hair,"[3, 7, 16, 18, 20, 21, 26, 29, 65, 71, 72, 77,...",78,"[1018, 21, 167, 696, 1055, 277, 260, 412, 396,...",10,0.128205,0.128205,1.0
7,conference table,"[348, 1075]",2,"[1044, 535, 1054, 200, 551, 855, 491, 404, 172...",0,0.000000,0.000000,0.0
8,wall,"[1, 3, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17,...",283,"[157, 995, 607, 720, 1044, 322, 144, 253, 285,...",7,0.024735,0.024735,0.7
9,building,"[1, 2, 9, 11, 16, 22, 23, 24, 25, 26, 29, 42, ...",385,"[56, 577, 649, 728, 774, 776, 981, 880, 602, 432]",10,0.025974,0.025974,1.0
